In [40]:
import logging
import boto3
import boto3.session
from datetime import date, datetime
from botocore.exceptions import ClientError
import json
import sys
import time
import os
import argparse


# http://boto3.readthedocs.io/en/latest/reference/services/ec2.html#service-resource

ec2 = boto3.resource('ec2')
client = boto3.client('ec2')

In [12]:
# session = boto3.session.Session()
REGION = "us-east-1"
cidr_block = '10.0.0.0/16'

# logger config
logger = logging.getLogger()
logging.basicConfig(level=logging.INFO,
                    format='%(asctime)s: %(levelname)s: %(message)s')

In [13]:
# create VPC
vpc = ec2.create_vpc(CidrBlock=cidr_block)
# we can assign a name to vpc, or any resource, by using tag
vpc.create_tags(Tags=[{"Key": "Name", "Value": "python-vpc"}])
vpc.wait_until_available()
print(vpc.id)

vpc-0bab92006ed749103


In [77]:
# Code below also creates a vpc (alternative to the above)
# ec2_client = session.client('ec2', REGION)
# ec2_resource = session.resource('ec2', REGION)
# create_vpc_response = ec2_client.create_vpc(CidrBlock=cidr_block)
# vpc = ec2_resource.Vpc(create_vpc_response["Vpc"]["VpcId"])

In [14]:
# create then attach internet gateway
ig = ec2.create_internet_gateway()
ig.create_tags(Tags=[{"Key": "Name", "Value": "python-vpc-igw"}])
#attach to vpc using the gateway id
vpc.attach_internet_gateway(InternetGatewayId=ig.id)
print(ig.id)

#create_internet_gateway also returns a dict
# create_ig_response = ec2_client.create_internet_gateway()
# ig_id = create_ig_response["InternetGateway"]["InternetGatewayId"]

igw-0af665ba3b76e7b59


## Create public and private subnets

In [15]:
cidr_blocks = ["10.0.1.0/24", "10.0.2.0/24", "10.0.3.0/24", "10.0.4.0/24"]
availability_zones = ["us-east-1a", "us-east-1b"]

# create subnet
#define a create subnet function
def public_private_subnets(cidr_list, azs, vpc_id):
    """input: cidr_block list, availability zones' list and vpc_id
    output: list of public and private subnets"""
    #initialize empty list for subnets
    subnets = []
    for idx in range(len(cidr_list)):
        #because I want a public and private SN to be in same az
        if idx % 2 == 0:
            subnet = ec2.create_subnet(CidrBlock=cidr_list[idx], VpcId=vpc_id, \
                AvailabilityZone = azs[0])
            subnets.append(subnet)
        elif idx % 2 == 1:
            subnet = ec2.create_subnet(CidrBlock=cidr_list[idx], VpcId=vpc_id, \
                AvailabilityZone = azs[1])            
            subnets.append(subnet)
    #create tags
    for idx in range(len(subnets)):
        if idx % 2 == 0:
            subnets[idx].create_tags(Tags=[{"Key": "Name", "Value": "publicSN_" + str(idx+1)}])
        else:
            subnets[idx].create_tags(Tags=[{"Key": "Name", "Value": "privateSN_" + str(idx)}])

    return subnets

subnets = public_private_subnets(cidr_blocks, availability_zones, vpc.id)

for subnet in subnets:
    print(subnet.id)

#subnet = ec2.create_subnet(CidrBlock='192.168.1.0/24', VpcId=vpc.id)
#or 
#create_subnet can be run directly on the vpc object
# subnet = vpc.create_subnet(CidrBlock=subnet_cidr, AvailabilityZone="{}{}".format(REGION, az))

subnet-0fb7251d8c6a66f37
subnet-09bfa64c471712272
subnet-01ba90a979e2f13c5
subnet-0f3b5f819da843367


In [16]:
def wait_nat_creation(nat_gateway_id):
    """
    Check if successful state is reached every 15 seconds until a successful state is reached.
    An error is returned after 40 failed checks.
    """
    try:
        waiter = client.get_waiter('nat_gateway_available')
        waiter.wait(NatGatewayIds=[nat_gateway_id])
    except ClientError:
        logger.exception(f'Could not create the NAT gateway.')
        raise

In [17]:
#create NAT gateway 
#initialize elastic ip for same
allocation = client.allocate_address(Domain='vpc')
# response = ec2.associate_address(AllocationId=allocation['AllocationId'],
#                                     InstanceId='INSTANCE_ID')
# print(response)
nat_gw = client.create_nat_gateway(SubnetId=subnets[0].id, AllocationId=allocation['AllocationId'],
            TagSpecifications=[{
                'ResourceType':
                'natgateway',
                'Tags': [{
                    'Key': 'Name',
                    'Value': 'python-nat-gateway'
                }]
            }])

nat_gw_id = nat_gw['NatGateway']['NatGatewayId']

wait_nat_creation(nat_gw_id)

In [18]:
print(nat_gw_id)

nat-0c761b9d492982804


## Create route tables and associations

In [81]:
# # create a public route table, this can be used when creating a single route table

# public_route_table = vpc.create_route_table()
# # and a public route, assign public cidr block and attach to internet gateway created above
# route = public_route_table.create_route(
#     DestinationCidrBlock='0.0.0.0/0',
#     GatewayId=ig.id
# )
# # associate the route table with the subnet
# public_route_table.associate_with_subnet(SubnetId=subnet.id)

# print(public_route_table.id)

## Code below for multiple route tables


In [20]:
# define a function to create more than one route table at a time

def route_tables(rt_titles, gw_list):
    """input: cidr_block list, availability zones' list and vpc_id
    output: list of public and private subnets"""
    #initialize empty route table list to collect route_tables
    route_tables = []
    for idx in range(len(rt_titles)):

        title = vpc.create_route_table()

        # and a public route, assign public cidr block and attach to internet gateway created above
        g_way = gw_list[idx]
        if type(g_way) is dict:

            route = title.create_route(
                DestinationCidrBlock='0.0.0.0/0',
                GatewayId=g_way['NatGateway']['NatGatewayId']
            )
        # associate the route table with the subnet
        # for idx2 in range(len(rt_titles)):
            title.associate_with_subnet(SubnetId=subnets[idx].id)
            idx2 = idx + 2
            title.associate_with_subnet(SubnetId=subnets[idx2].id)

            route_tables.append(title)
        else:

            route = title.create_route(
                DestinationCidrBlock='0.0.0.0/0',
                GatewayId=g_way.id
            )
        # associate the route table with the subnet
        # for idx2 in range(len(rt_titles)):
            title.associate_with_subnet(SubnetId=subnets[idx].id)
            idx2 = idx + 2
            title.associate_with_subnet(SubnetId=subnets[idx2].id)

            route_tables.append(title)

    #iterate to create tags for route tables
    for idx in range(len(route_tables)):
        route_tables[idx].create_tags(Tags=[{"Key": "Name", "Value": str(rt_titles[idx])}])

    return route_tables



In [21]:
#initialize route table variables
rt_titles = ["public_route_table", "private_route_table"]
gws = [ig, nat_gw]

#call route table function to create public and private route tables
rt_tables = route_tables(rt_titles, gws)

print(rt_tables)

[ec2.RouteTable(id='rtb-0839a8dcf82616d33'), ec2.RouteTable(id='rtb-00fe78c675a4ae18e')]


## Create frontend and backend security groups

In [22]:
# Create frontend sec group
frontend_sg = ec2.create_security_group(
    GroupName='frontend_sg_py_vpc', Description='front end security group', VpcId=vpc.id)

In [23]:
#attach front end security permissions
frontend_sg.authorize_ingress(IpPermissions = [
    {
    'IpProtocol':'tcp',
    'FromPort':443,
    'ToPort':443,
    'IpRanges':[{'CidrIp':'0.0.0.0/0'}]
    },
    {
    'IpProtocol':'tcp',
    'FromPort':80,
    'ToPort':80,
    'IpRanges':[{'CidrIp':'0.0.0.0/0'}]
    },
    {
    'IpProtocol':'tcp',
    'FromPort':22,
    'ToPort':22,
    'IpRanges':[{'CidrIp':'0.0.0.0/0'}]
    },
]
)

# frontend_sg.authorize_egress(IpPermissions = [
#     {
#     'IpProtocol':'-1',
#     'FromPort':0,
#     'ToPort':0,
#     'IpRanges':[{'CidrIp':'0.0.0.0/0'}]
#     }
# ]
# )



{'Return': True,
 'SecurityGroupRules': [{'SecurityGroupRuleId': 'sgr-0828d8b3fdfa628ea',
   'GroupId': 'sg-0ece1da816fd49009',
   'GroupOwnerId': '957171820283',
   'IsEgress': False,
   'IpProtocol': 'tcp',
   'FromPort': 443,
   'ToPort': 443,
   'CidrIpv4': '0.0.0.0/0'},
  {'SecurityGroupRuleId': 'sgr-0aa1c008a7710ce70',
   'GroupId': 'sg-0ece1da816fd49009',
   'GroupOwnerId': '957171820283',
   'IsEgress': False,
   'IpProtocol': 'tcp',
   'FromPort': 80,
   'ToPort': 80,
   'CidrIpv4': '0.0.0.0/0'},
  {'SecurityGroupRuleId': 'sgr-0f3fd03c981b539b6',
   'GroupId': 'sg-0ece1da816fd49009',
   'GroupOwnerId': '957171820283',
   'IsEgress': False,
   'IpProtocol': 'tcp',
   'FromPort': 22,
   'ToPort': 22,
   'CidrIpv4': '0.0.0.0/0'}],
 'ResponseMetadata': {'RequestId': '74ac05df-ad8e-4f53-98c3-3e29f4461b11',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '74ac05df-ad8e-4f53-98c3-3e29f4461b11',
   'cache-control': 'no-cache, no-store',
   'strict-transport-security': '

In [24]:
#Create backend security group
backend_sg = ec2.create_security_group(
    GroupName='backend_sg_py_vpc', Description='back end security group', VpcId=vpc.id)

In [25]:
##attach back end security permissions

backend_sg.authorize_ingress(IpPermissions = [    
    {
    'IpRanges':[{'CidrIp':"10.0.1.0/24", 'CidrIp':"10.0.2.0/24"}],
    'IpProtocol':'tcp',
    'FromPort':3306,
    'ToPort':3306
    },
    {
    'IpRanges':[{'CidrIp':"10.0.1.0/24", 'CidrIp':"10.0.2.0/24"}],
    'IpProtocol':'tcp',
    'FromPort':22,
    'ToPort':22
    }
]
)
# egress already created with sec group
# backend_sg.authorize_egress(IpPermissions = [
#     {    
#     'IpRanges':[{'CidrIp':'0.0.0.0/0'}],
#     'IpProtocol':'-1',
#     'FromPort':0,
#     'ToPort':0
#     }
# ]
# )


{'Return': True,
 'SecurityGroupRules': [{'SecurityGroupRuleId': 'sgr-0ca8e365bed191891',
   'GroupId': 'sg-0c3884aeb6c361459',
   'GroupOwnerId': '957171820283',
   'IsEgress': False,
   'IpProtocol': 'tcp',
   'FromPort': 3306,
   'ToPort': 3306,
   'CidrIpv4': '10.0.2.0/24'},
  {'SecurityGroupRuleId': 'sgr-0eccddac02afc650c',
   'GroupId': 'sg-0c3884aeb6c361459',
   'GroupOwnerId': '957171820283',
   'IsEgress': False,
   'IpProtocol': 'tcp',
   'FromPort': 22,
   'ToPort': 22,
   'CidrIpv4': '10.0.2.0/24'}],
 'ResponseMetadata': {'RequestId': '0402f0cc-7d25-47fe-93d0-2b4386d87444',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '0402f0cc-7d25-47fe-93d0-2b4386d87444',
   'cache-control': 'no-cache, no-store',
   'strict-transport-security': 'max-age=31536000; includeSubDomains',
   'content-type': 'text/xml;charset=UTF-8',
   'content-length': '1132',
   'date': 'Thu, 14 Apr 2022 00:49:54 GMT',
   'server': 'AmazonEC2'},
  'RetryAttempts': 0}}

In [26]:
print(frontend_sg.id, backend_sg.id)

sg-0ece1da816fd49009 sg-0c3884aeb6c361459


## Create S3 bucket and policy

In [27]:
# Create a bucket policy
bucket_name = 'py-med-bucket'

# s3_location = {
#     'LocationConstraint':'us-east-1'
# }
# , CreateBucketConfiguration=s3_location
bucket_policy = {
    'Version': '2012-10-17',
    'Statement': [{
        'Sid': 'PublicReadGetObject',
        'Effect': 'Allow',
        'Principal': '*',
        'Action': ['s3:GetObject', 's3:GetObjectVersion'],
        'Resource': f'arn:aws:s3:::{bucket_name}/*'
    }]
}

# Convert the policy from JSON dict to string
bucket_policy = json.dumps(bucket_policy)

# Set the new policy
s3 = boto3.client('s3')
s3.create_bucket(Bucket=bucket_name, ACL='public-read')
s3.put_bucket_policy(Bucket=bucket_name, Policy=bucket_policy)

{'ResponseMetadata': {'RequestId': '49B1HSAPZ7QHXF4P',
  'HostId': '3VG9IlrKv1JTwyk7a3OoJhqCz8XYC2ffrAsL5Yw2q2D/AM57Zk9LPSSeutsdoVKWFcnFua7Msms=',
  'HTTPStatusCode': 204,
  'HTTPHeaders': {'x-amz-id-2': '3VG9IlrKv1JTwyk7a3OoJhqCz8XYC2ffrAsL5Yw2q2D/AM57Zk9LPSSeutsdoVKWFcnFua7Msms=',
   'x-amz-request-id': '49B1HSAPZ7QHXF4P',
   'date': 'Thu, 14 Apr 2022 00:50:06 GMT',
   'server': 'AmazonS3'},
  'RetryAttempts': 0}}

## Create Apache web server instance with user data

In [ ]:
ec2_key = boto3.connect_ec2()
key = ec2_key = boto3.connect_ec2().create_key_pair('mynewkey')
key.save('~/OneDrive/Academia/Cloud_comput/Key_pairs/')

## Initialize Apache and SQL command line scripts

In [28]:
APACHE_SCRIPT = """#!/bin/bash
  #sudo yum update -y
  sudo yum install httpd -y
  sudo service httpd start
  sudo chkconfig httpd on
  cd /var/www/html
  echo "<html><h1>This is Apache Web Server 01</h1></html>" > index.html
  sudo yum install mysql -y
  """

SQLDB_SCRIPT = """#!/bin/bash
sudo yum update -y
sudo yum install mysql-server -y
sudo service mysqld start
"""

## Create Apache web server

In [29]:
# Create instance
apache_server = ec2.create_instances(
    ImageId='ami-0b0af3577fe5e3532', InstanceType='t2.micro', MaxCount=1, MinCount=1, KeyName='win_keypair',
    NetworkInterfaces=[{'SubnetId': subnets[0].id, 'DeviceIndex': 0, 'AssociatePublicIpAddress': True, 'Groups': [frontend_sg.group_id]}],
    UserData=APACHE_SCRIPT)

#wait until apache is running to provide output
apache_server[0].wait_until_running()

print(apache_server[0].id)

i-02ae2033b2216dd1a


## Create MySQL database server

In [30]:
# Create MySQL instance and assign to back end security group
sql_server = ec2.create_instances(
    ImageId='ami-0b0af3577fe5e3532', InstanceType='t2.micro', MaxCount=1, MinCount=1, KeyName='win_keypair',
    NetworkInterfaces=[{'SubnetId': subnets[1].id, 'DeviceIndex': 0, 'AssociatePublicIpAddress': True, 'Groups': [backend_sg.group_id]}],
    UserData=SQLDB_SCRIPT)

#wait until apache is running to provide output
sql_server[0].wait_until_running()

print(sql_server[0].id)

i-06aa1aac366d567d2


## Destroy VPC

In [ ]:
#!/usr/bin/env python
#this portion was completely sourced from the github link below
#https://gist.github.com/vernhart/c6a0fc94c0aeaebe84e5cd6f3dede4ce

"""I was trying to programatically remove a Virtual Private Cloud (VPC) in
AWS and the error message was not helpful:
    botocore.exceptions.ClientError: An error occurred (DependencyViolation)
    when calling the DeleteVpc operation: The vpc 'vpc-c12029b9' has
    dependencies and cannot be deleted.
Searching for a quick solution was not fruitful but I was able to glean some
knowledge from Neil Swinton's gist:
https://gist.github.com/neilswinton/d37787a8d84387c591ff365594bd26ed
Using that, and some trial and error, I was able to develop this function
that does all the cleanup necessary.
Word of warning: This will delete the VPC and all instances/resources
associated with it. As far as I know, this is complete. It's just like
selecting Delete from the context menu on a VPC in the AWS Console except
that this also deletes internet gateways that are attached to the VPC.
"""


def vpc_cleanup(vpcid):
    """Remove VPC from AWS
    Set your region/access-key/secret-key from env variables or boto config.
    :param vpcid: id of vpc to delete
    """
    if not vpcid:
        return
    print('Removing VPC ({}) from AWS'.format(vpcid))
    ec2 = boto3.resource('ec2')
    ec2client = ec2.meta.client
    vpc = ec2.Vpc(vpcid)
    # detach and delete all gateways associated with the vpc
    for gw in vpc.internet_gateways.all():
        vpc.detach_internet_gateway(InternetGatewayId=gw.id)
        gw.delete()
    # delete all route table associations
    for rt in vpc.route_tables.all():
        for rta in rt.associations:
            if not rta.main:
                rta.delete()
    # delete any instances
    for subnet in vpc.subnets.all():
        for instance in subnet.instances.all():
            instance.terminate()
    # delete our endpoints
    for ep in ec2client.describe_vpc_endpoints(
            Filters=[{
                'Name': 'vpc-id',
                'Values': [vpcid]
            }])['VpcEndpoints']:
        ec2client.delete_vpc_endpoints(VpcEndpointIds=[ep['VpcEndpointId']])
    # delete our security groups
    for sg in vpc.security_groups.all():
        if sg.group_name != 'default':
            sg.delete()
    # delete any vpc peering connections
    for vpcpeer in ec2client.describe_vpc_peering_connections(
            Filters=[{
                'Name': 'requester-vpc-info.vpc-id',
                'Values': [vpcid]
            }])['VpcPeeringConnections']:
        ec2.VpcPeeringConnection(vpcpeer['VpcPeeringConnectionId']).delete()
    # delete non-default network acls
    for netacl in vpc.network_acls.all():
        if not netacl.is_default:
            netacl.delete()
    # delete network interfaces
    for subnet in vpc.subnets.all():
        for interface in subnet.network_interfaces.all():
            interface.delete()
        subnet.delete()
    # finally, delete the vpc
    ec2client.delete_vpc(VpcId=vpcid)


def main(argv=None):
    vpc_cleanup(vpc.id)


if __name__ == '__main__':
    main(sys.argv)

In [48]:
def destroy_ec2(vpc_id, aws_region):
    logger.debug(f"{vpc_id}")
    ec2 = boto3.resource("ec2", region_name=aws_region)
    ec2client = ec2.meta.client
    # test for valid credentials
    try:
        ec2client.describe_vpcs()
    except ClientError as e:
        logging.info(e)
        print(
            "Either your credentials are invalid or your IAM user doesn't have permissions to list VPCs"
        )
        sys.exit(1)

    if not vpc_exists(ec2client, vpc_id):
        print(f"VPC {vpc_id} does not exist in {aws_region}")
        return

    vpc = ec2.Vpc(vpc_id)

    # disassociate EIPs and release EIPs from EC2 instances
    for subnet in vpc.subnets.all():
        for instance in subnet.instances.all():
            filters = [{"Name": "instance-id", "Values": [instance.id]}]
            eips = ec2client.describe_addresses(Filters=filters)["Addresses"]
            for eip in eips:
                ec2client.disassociate_address(AssociationId=eip["AssociationId"])
                ec2client.release_address(AllocationId=eip["AllocationId"])

    # delete instances
    filters = [
        {"Name": "instance-state-name", "Values": ["running"]},
        {"Name": "vpc-id", "Values": [vpc_id]},
    ]
    ec2_instances = ec2client.describe_instances(Filters=filters)
    instance_ids = []
    for reservation in ec2_instances["Reservations"]:
        instance_ids += [
            instance["InstanceId"] for instance in reservation["Instances"]
        ]

    logger.info(f"instance deletion list: {instance_ids}")
    if instance_ids:
        logging.info("Waiting for instances to terminate")
        waiter = ec2client.get_waiter("instance_terminated")
        ec2client.terminate_instances(InstanceIds=instance_ids)
        waiter.wait(InstanceIds=instance_ids)


def destroy_services(vpc_id, aws_region, services):
    services_map = {"ec2": destroy_ec2}

    for service in services.split(","):
        try:
            services_map[service](vpc_id, aws_region)
        except KeyError:
            logger.error(f"destroying {service} not implemented")


def vpc_exists(ec2client, vpc_id):
    try:
        ec2client.describe_vpcs(VpcIds=[vpc_id])
    except ClientError as e:
        logging.info(e)
        return False
    return True


def delete_vpc(vpc_id, aws_region, release_eips=False):
    ec2 = boto3.resource("ec2", region_name=aws_region)
    ec2client = ec2.meta.client
    if not vpc_exists(ec2client, vpc_id):
        print(f"VPC {vpc_id} does not exist in {aws_region}")
        return False

    # Exit cleanly if user did to specify at command line to delete EC2 instances for
    # a VPC with runnining instances
    filters = [
        {"Name": "instance-state-name", "Values": ["running"]},
        {"Name": "vpc-id", "Values": [vpc_id]},
    ]
    if ec2client.describe_instances(Filters=filters)["Reservations"]:
        print(
            f"Running EC2 instances exist in {vpc_id}. Please use --services ec2 to invoke the program."
        )
        return False

    vpc = ec2.Vpc(vpc_id)

    # delete transit gateway attachment for this vpc
    # note - this only handles vpc attachments, not vpn
    for attachment in ec2client.describe_transit_gateway_attachments()[
        "TransitGatewayAttachments"
    ]:
        if attachment["ResourceId"] == vpc_id:
            ec2client.delete_transit_gateway_vpc_attachment(
                TransitGatewayAttachmentId=attachment["TransitGatewayAttachmentId"]
            )

    # delete NAT Gateways
    # attached ENIs are automatically deleted
    # EIPs are disassociated but not released
    filters = [{"Name": "vpc-id", "Values": [vpc_id]}]
    for nat_gateway in ec2client.describe_nat_gateways(Filters=filters)["NatGateways"]:
        ec2client.delete_nat_gateway(NatGatewayId=nat_gateway["NatGatewayId"])

    # detach default dhcp_options if associated with the vpc
    dhcp_options_default = ec2.DhcpOptions("default")
    if dhcp_options_default:
        dhcp_options_default.associate_with_vpc(VpcId=vpc.id)

    # delete any vpc peering connections
    for vpc_peer in ec2client.describe_vpc_peering_connections()[
        "VpcPeeringConnections"
    ]:
        if vpc_peer["AccepterVpcInfo"]["VpcId"] == vpc_id:
            ec2.VpcPeeringConnection(vpc_peer["VpcPeeringConnectionId"]).delete()
        if vpc_peer["RequesterVpcInfo"]["VpcId"] == vpc_id:
            ec2.VpcPeeringConnection(vpc_peer["VpcPeeringConnectionId"]).delete()

    # delete our endpoints
    for ep in ec2client.describe_vpc_endpoints(
        Filters=[{"Name": "vpc-id", "Values": [vpc_id]}]
    )["VpcEndpoints"]:
        ec2client.delete_vpc_endpoints(VpcEndpointIds=[ep["VpcEndpointId"]])

    # delete custom security groups
    for sg in vpc.security_groups.all():
        if sg.group_name != "default":
            sg.delete()

    # delete custom NACLs
    for netacl in vpc.network_acls.all():
        if not netacl.is_default:
            netacl.delete()

    # ensure ENIs are deleted before proceding
    timeout = time.time() + 300
    filter = [{"Name": "vpc-id", "Values": [vpc_id]}]
    logger.debug(f"proceed with deleting ENIs")
    reached_timeout = True
    while time.time() < timeout:
        if not ec2client.describe_network_interfaces(Filters=filters)[
            "NetworkInterfaces"
        ]:
            logger.info(f"no ENIs remaining")
            reached_timeout = False
            break
        else:
            logger.info(f"waiting on ENIs to delete")
            time.sleep(30)

    if reached_timeout:
        logger.debug(f"ENI deletion timed out")

    # delete subnets
    for subnet in vpc.subnets.all():
        for interface in subnet.network_interfaces.all():
            interface.delete()
        subnet.delete()

    # Delete routes, associations, and routing tables
    filter = [{"Name": "vpc-id", "Values": [vpc_id]}]
    route_tables = ec2client.describe_route_tables(Filters=filter)["RouteTables"]
    for route_table in route_tables:
        for route in route_table["Routes"]:
            if route["Origin"] == "CreateRoute":
                ec2client.delete_route(
                    RouteTableId=route_table["RouteTableId"],
                    DestinationCidrBlock=route["DestinationCidrBlock"],
                )
            for association in route_table["Associations"]:
                if not association["Main"]:
                    ec2client.disassociate_route_table(
                        AssociationId=association["RouteTableAssociationId"]
                    )
                    ec2client.delete_route_table(
                        RouteTableId=route_table["RouteTableId"]
                    )
    # delete routing tables without associations
    for route_table in route_tables:
        if route_table["Associations"] == []:
            ec2client.delete_route_table(RouteTableId=route_table["RouteTableId"])

    # destroy NAT gateways
    filters = [{"Name": "vpc-id", "Values": [vpc_id]}]
    nat_gateway_ids = [
        nat_gateway["NatGatewayId"]
        for nat_gateway in ec2client.describe_nat_gateways(Filters=filters)[
            "NatGateways"
        ]
    ]
    for nat_gateway_id in nat_gateway_ids:
        ec2client.delete_nat_gateway(NatGatewayId=nat_gateway_id)

    # detach and delete all IGWs associated with the vpc
    for gw in vpc.internet_gateways.all():
        vpc.detach_internet_gateway(InternetGatewayId=gw.id)
        gw.delete()

    ec2client.delete_vpc(VpcId=vpc_id)
    return True


# logger.info(f"calling delete_vpc with {vpc_id}")
# if delete_vpc(vpc_id=vpc_id, aws_region=aws_region, release_eips=False):
#     print(f"destroyed {vpc_id} in {aws_region}")
# else:
#     print(f"unable to destroy {vpc_id} in {aws_region}")

In [49]:
destroy_ec2("vpc-0bab92006ed749103", REGION)



2022-04-13 21:29:12,409: INFO: instance deletion list: ['i-02ae2033b2216dd1a', 'i-06aa1aac366d567d2']
2022-04-13 21:29:12,410: INFO: Waiting for instances to terminate


In [50]:
delete_vpc("vpc-0bab92006ed749103", REGION, release_eips=False)

2022-04-13 21:30:00,028: INFO: waiting on ENIs to delete
2022-04-13 21:30:30,164: INFO: waiting on ENIs to delete
2022-04-13 21:31:00,417: INFO: waiting on ENIs to delete
2022-04-13 21:31:30,574: INFO: waiting on ENIs to delete
2022-04-13 21:32:00,701: INFO: waiting on ENIs to delete
2022-04-13 21:32:30,818: INFO: waiting on ENIs to delete
2022-04-13 21:33:00,998: INFO: no ENIs remaining


True